Notebook for preprocessing d3 datasets.

In [1]:
import os
import json
from os.path import join
import pandas as pd
from tqdm import tqdm
from collections import defaultdict

Download the bbbp dataset (csv)

In [5]:
df = pd.read_csv('galactica_raw/bbbp.csv')

In [6]:
df['outcome'] = df['p_np']

In [7]:
df.head()

,num,name,p_np,smiles,outcome
0,1,Propanolol,1,[Cl].CC(C)NCC(O)COc1cccc2ccccc12,1
1,2,Terbutylchlorambucil,1,C(=O)(OC(C)(C)C)CCCc1ccc(cc1)N(CCCl)CCCl,1
2,3,40730,1,c12c3c(N4CCN(C)CC4)c(F)cc1c(c(C(O)=O)cn2C(C)CO...,1
3,4,24,1,C1CCN(CC1)Cc1cccc(c1)OCCCNC(=O)C,1
4,5,cloxacillin,1,Cc1onc(c2ccccc2Cl)c1C(=O)N[C@H]3[C@H]4SC(C)(C)...,1


In [15]:
df.name.loc[df.name.str.isnumeric()] = 'Compound-'+df.name.loc[df.name.str.isnumeric()]

/tmp/ipykernel_3655676/2615518426.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.name.loc[df.name.str.isnumeric()] = 'Compound-'+df.name.loc[df.name.str.isnumeric()]


Download the raw tox21 dataset (csv)

In [11]:
df = pd.read_csv('moleculenet_raw/tox21.csv')
smiles = df['smiles']
d = df.drop(columns=['mol_id', 'smiles'])

In [13]:
d.head()

,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53
0,0.0,0.0,1.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
d.sum(axis=1).value_counts()

0.0    4959
1.0    1398
2.0     670
3.0     404
4.0     208
5.0     107
6.0      56
7.0      24
8.0       4
9.0       1
dtype: int64

In [16]:
d.sum(axis=0).value_counts()

309.0    1
237.0    1
768.0    1
300.0    1
793.0    1
350.0    1
186.0    1
942.0    1
264.0    1
372.0    1
918.0    1
423.0    1
dtype: int64

In [ ]:
N = len(data)
task_names = ['d3_' + str(i) for i in range(N)]
task_defs = {
    task_names[i]: data[i]['human_annotations'][0]
    for i in range(N)
}
for i, task_name in enumerate(task_names):
    inputs = data[i]['positive_samples'] + data[i]['negative_samples']
    outputs = ["yes"] * len(data[i]['positive_samples']) + ["no"] * len(data[i]['negative_samples'])
    df = pd.DataFrame.from_dict({
        'input': inputs,
        'output': outputs,
    })

    df['text'] = 'Input: ' + df['input'] + ' Answer: ' + df['output'] + '\n'
    print(task_defs[task_name])
    print(df.iloc[0].text)
    print('\n\n')
    df.to_csv(join(out_dir, task_name + '.csv'), index=False)
json.dump(task_defs, open(join(out_dir, 'task_defs.json'), 'w'), indent=4)

In [ ]:
for i in range(N):
    print("'" + task_names[i] + "': {\n\t'check_answer_func': r'" + ','.join(data[i]['human_annotations']) + "'\n},\n")
# 'synonyms': {
#         'check_answer_func': r'synonym|alternate|rephrase',
#     },